In [1]:
import torch
torch.manual_seed(2)
a = torch.rand(3,3)

In [2]:
a

tensor([[0.6147, 0.3810, 0.6371],
        [0.4745, 0.7136, 0.6190],
        [0.4425, 0.0958, 0.6142]])

In [3]:
# adjacent matrix
a[a>0.5] = 1
a[a<0.5] = 0
a

tensor([[1., 0., 1.],
        [0., 1., 1.],
        [0., 0., 1.]])

In [24]:
a.sum(dim=-1)

tensor([2., 2., 1.])

In [4]:
# degree matrix
def Degree_matrix(a):
    return torch.diag(a.sum(dim=1))
d = Degree_matrix(a)
d

tensor([[2., 0., 0.],
        [0., 2., 0.],
        [0., 0., 1.]])

In [26]:
# laplacian matrix
# L = Degree matrix - Adjacent matrix
def laplacian(a):
    return Degree_matrix(a)-a
l = laplacian(a)
l

tensor([[ 1.,  0., -1.],
        [ 0.,  1., -1.],
        [ 0.,  0.,  0.]])

In [27]:
#
def calc_degree_matrix_norm(a):
    return torch.diag(torch.pow(a.sum(dim=-1),-0.5))

In [28]:
calc_degree_matrix_norm(a)

tensor([[0.7071, 0.0000, 0.0000],
        [0.0000, 0.7071, 0.0000],
        [0.0000, 0.0000, 1.0000]])

In [39]:
# L_norm = I - (D^-0.5 * A * D^-0.5)
def create_graph_lapl_norm(a):
 size = a.shape[-1]
 D_norm = calc_degree_matrix_norm(a)
 L_norm = torch.ones(size) - (D_norm @ a @ D_norm )
 return L_norm

In [40]:
create_graph_lapl_norm(a)

tensor([[0.5000, 1.0000, 0.2929],
        [1.0000, 0.5000, 0.2929],
        [1.0000, 1.0000, 0.0000]])

In [5]:
class Student_Grade: 
    def __init__(self, name, grade):  
        self.name = name
        self.grade = grade

    def print_grade(self):
        print("%s grade is %s" % (self.name,self.grade))

s1 = Student_Grade("Tom", 8)  # 创建对象s1，实例化直接传入参数
s2 = Student_Grade("Jerry", 7)  # 创建对象s2

s1.print_grade()
s2.print_grade()



Tom grade is 8
Jerry grade is 7


In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

def device_as(x,y):
  return x.to(y.device)

# tensor operations now support batched inputs
def calc_degree_matrix_norm(a):
  return torch.diag_embed(torch.pow(a.sum(dim=-1),-0.5))

def create_graph_lapl_norm(a):
  size = a.shape[-1]
  a +=  device_as(torch.eye(size),a)
  D_norm = calc_degree_matrix_norm(a)
  L_norm = torch.bmm( torch.bmm(D_norm, a) , D_norm )
  return L_norm

class GCN_AISUMMER(nn.Module):
    """
    A simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    def __init__(self, in_features, out_features, bias=True):
    super().__init__()
    self.linear = nn.Linear(in_features, out_features, bias=bias)

    def forward(self, X, A):
        """
        A: adjecency matrix
        X: graph signal
        """
        L = create_graph_lapl_norm(A)
        x = self.linear(X)
        return torch.bmm(L, x)